In [2]:
# import packages
import warnings
warnings.filterwarnings('ignore')
import cv2
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [3]:
# getting the cifar10 dataset from keras
(X_train, y_train), (X_test, y_test) = cifar10.load_data() 

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 451s 3us/step


In [4]:
# split the dataset into training and validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.1, random_state=0)

In [5]:
print(f"Train Image Shape, {X_train.shape()}")
print(f"Train labels Shape, {y_train.shape()}")

print(f"Validation Image Shape, {X_valid.shape()}")
print(f"Validation labels Shape, {y_valid.shape()}")

print(f"Test Image Shape, {x_test.shape()")
print(f"Test labels Shape, {y_test.shape()")

SyntaxError: f-string: expecting '}' (826713927.py, line 7)